# Post Processing

This notebook demonstrates how to use `history.postprocessing` module. For more details on the Post-Processing [README](README.md). You can also use the prefect version by using `history.postprocessing_prefect` module.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import history.postprocessing.pipeline as pp

# Prefect version
# import history.postprocessing_prefect.flows as ppfl
# from prefect.task_runners import ConcurrentTaskRunner

## ⚙️ Settings and Path Management

Main directories used in the post-processing pipeline:

- **`raw_dir`** → Contains all raw input archives.  
- **`extracted_dir`** → Stores extracted archives.  
- **`processing_dir`** → Main workspace for intermediate and processed data.  
- **`plots_dir`** → Output folder for generated plots and visualizations.

> 💡 These directories are automatically created when running the Prefect flows.


In [ ]:
# paths
raw_dir = "/mnt/summer/USERS/DEHECQA/history/output/raw"
extracted_dir = "/mnt/summer/USERS/DEHECQA/history/output/extracted"
processing_dir = pp.ProcessingDirectory("/mnt/summer/USERS/DEHECQA/history/output/processing" )
plots_dir = "/mnt/summer/USERS/DEHECQA/history/output/plots" 

# other settings
OVERWRITE = False
DRY_RUN = False # set this to True to avoid process
PDAL_EXEC_PATH = "/home/godinlu/micromamba/envs/pdal/bin/pdal"
MAX_WORKERS = 4

### 🧩 Step 1 — Extract archives

In [ ]:
pp.uncompress_all_submissions(raw_dir, extracted_dir, OVERWRITE)

# Prefect Version
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.uncompress_all_submissions.with_options(task_runner=task_runner)(raw_dir, extracted_dir, OVERWRITE)

### 🧩 Step 2 — Analyse submissions and create symlinks

In [ ]:
pp.index_submissions_and_link_files(extracted_dir, processing_dir)

### 🧩 Step 3 — Create auxiliary data symlinks

In [ ]:
aux_data_mapping = {
    ("casa_grande", "aerial") : [
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_ref_dem_zoom_5m.tif",
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_ref_dem_zoom_5m_mask.tif",
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_landcover_zoom.tif"
    ],
    ("casa_grande", "kh9mc") : [
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_ref_dem_large.tif",
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_ref_dem_large_mask.tif",
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_landcover_large.tif"
    ],
    ("iceland", "aerial") : [
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_ref_dem_zoom_5m.tif",
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_ref_dem_zoom_5m_mask.tif",
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_landcover_zoom.tif"
    ],
    ("iceland", "kh9mc") : [
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_ref_dem_large.tif",
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_ref_dem_large_mask.tif",
        "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_landcover_large.tif"
    ]
}

# Same references dem for KH-9 MC and KH-9 PC
aux_data_mapping[("casa_grande", "kh9pc")] = aux_data_mapping[("casa_grande", "kh9mc")]
aux_data_mapping[("iceland", "kh9pc")] = aux_data_mapping[("iceland", "kh9mc")]

processing_dir.set_aux_files_symlinks(aux_data_mapping)

### 🧩 Step 4 — Convert point clouds to DEMs

In [ ]:
pp.process_pointclouds_to_dems(processing_dir, PDAL_EXEC_PATH, OVERWRITE, DRY_RUN)

# Prefect Version
#task_runner = ConcurrentTaskRunner(max_workers=4)
#ppfl.process_pointclouds_to_dems.with_options(task_runner=task_runner)(processing_dir, PDAL_EXEC_PATH, OVERWRITE, DRY_RUN)

### 🧩 Step 5 — Coregister DEMs

In [ ]:
pp.process_coregister_dems(processing_dir, OVERWRITE)

# Prefect version
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_coregister_dems.with_options(task_runner=task_runner)(processing_dir, OVERWRITE)

### 🧩 Step 6 — Generate Differential DEMs (DDEMs)


In [ ]:
pp.process_generate_ddems(processing_dir, OVERWRITE)

# Prefect Version
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_generate_ddems.with_options(task_runner=task_runner)(processing_dir, OVERWRITE)

### 🧩 Step 7 — Compute general statistics

In [ ]:
pp.process_compute_statistics(processing_dir)

# Prefect version
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_compute_statistics.with_options(task_runner=task_runner)(processing_dir)

### 🧩 Step 8 — Compute landcover-based statistics

In [ ]:
pp.process_compute_landcover_statistics(processing_dir)

# Prefect Version
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_compute_landcover_statistics.with_options(task_runner=task_runner)(processing_dir)

### 🧩 Step 9 — Generate STD DEMs

In [ ]:
pp.process_generate_std_dems(processing_dir, OVERWRITE)

# Prefect Version
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_generate_std_dems.with_options(task_runner=task_runner)(processing_dir, OVERWRITE)

### 🧩 Step 10 — Compute landcover-based statistics on STD DEMs

In [ ]:
pp.process_compute_landcover_statistics_on_std_dems(processing_dir)

# Prefect Version
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.process_compute_landcover_statistics_on_std_dems.with_options(task_runner=task_runner)(processing_dir)

### 🧩 Step 11 — Generate visualizations

In [ ]:
pp.generate_postprocessing_plots(processing_dir, plots_dir, MAX_WORKERS)

# Prefect Version
# task_runner = ConcurrentTaskRunner(max_workers=4)
# ppfl.generate_postprocessing_plots.with_options(task_runner=task_runner)(processing_dir, plots_dir)

## Visualize processing files

In [ ]:
processing_dir = pp.ProcessingDirectory(processing_dir)
processing_dir.submissions_summary()
processing_dir.plot()
